# MERGE DATASET - API DATA

### Libreries used

In [318]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import re
from sqlalchemy import create_engine
import json
import sys
import os

## Connection to database

In [319]:
sys.path.append('../otherconfig/')
from dbconfig import configuration

config = configuration('../otherconfig/database.ini')
db_url = f"postgresql+psycopg2://{config['user']}:{config['password']}@{config['host']}/{config['database']}"
engine = create_engine(db_url)

try:
    conn = engine.connect()
    print("Database connection successful.")
except Exception as e:
    print("Database connection failed:", e)
    sys.exit(1)


Reading configuration from ../otherconfig/database.ini
Database connection successful.


Next, this part will be for the practical use of the dags and the follow-up of the steps.

### Load the CSV data into a DataFrame

In [320]:
metacritic_data_path = '../data/metacritic_data_test1.csv'
try:
    metacritic_data = pd.read_csv(metacritic_data_path)
    print(f"CSV data loaded successfully with {len(metacritic_data)} records.")
except Exception as e:
    print("Failed to load CSV data:", e)
    sys.exit(1)

CSV data loaded successfully with 14055 records.


### Load data into PostgreSQL database

In [321]:
try:
    metacritic_data.to_sql('metacritic_data', conn, if_exists='replace', index=False)
    print("Data loaded into the PostgreSQL database successfully.")
except Exception as e:
    print("Failed to load data into the PostgreSQL database:", e)
    sys.exit(1)

Data loaded into the PostgreSQL database successfully.


## API DATA

#### List of CSV files

In [322]:
api_data_path = '../data/'
csv_files = [os.path.join(api_data_path, f'api_data_{i}.csv') for i in range(1, 14)]

#### Store DataFrames

In [323]:
dataframes = []
for file in csv_files:
    try:
        df = pd.read_csv(file)
        dataframes.append(df)
        print(f"Loaded {file} successfully with {len(df)} records.")
    except Exception as e:
        print(f"Failed to load {file}:", e)
        sys.exit(1)

Loaded ../data/api_data_1.csv successfully with 1000 records.
Loaded ../data/api_data_2.csv successfully with 1000 records.
Loaded ../data/api_data_3.csv successfully with 900 records.
Loaded ../data/api_data_4.csv successfully with 240 records.
Loaded ../data/api_data_5.csv successfully with 1000 records.
Loaded ../data/api_data_6.csv successfully with 1000 records.
Loaded ../data/api_data_7.csv successfully with 1000 records.
Loaded ../data/api_data_8.csv successfully with 1000 records.
Loaded ../data/api_data_9.csv successfully with 1000 records.
Loaded ../data/api_data_10.csv successfully with 1000 records.
Loaded ../data/api_data_11.csv successfully with 1000 records.
Loaded ../data/api_data_12.csv successfully with 1000 records.
Loaded ../data/api_data_13.csv successfully with 1000 records.


#### Concatenate all DataFrames into a single DataFrame

In [324]:
combined_api_data = pd.concat(dataframes, ignore_index=True)
print(f"Combined API data has {len(combined_api_data)} records.")

Combined API data has 12140 records.


In [325]:
combined_api_data.to_csv('../data/api_data1.csv', index=False)
print("Combined API data saved to 'api_data.csv'")

Combined API data saved to 'api_data.csv'


## General description

In [326]:
combined_api_data.head()

,id,slug,name,released,tba,background_image,rating,rating_top,ratings,ratings_count,...,dominant_color,platforms,parent_platforms,genres,stores,clip,tags,esrb_rating,short_screenshots,community_rating
0,9907,pictopix,Pictopix,2017-01-05,False,https://media.rawg.io/media/screenshots/ff0/ff...,3.96,4,"[{'id': 4, 'title': 'recommended', 'count': 20...",24,...,0f0f0f,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 40, 'name': 'Casual', 'slug': 'casual'...","[{'id': 10805, 'store': {'id': 1, 'name': 'Ste...",NaN,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...",NaN,"[{'id': -1, 'image': 'https://media.rawg.io/me...",NaN
1,4119,chronovolt,Chronovolt,2012-11-21,False,https://media.rawg.io/media/games/8dc/8dc50d62...,2.55,3,"[{'id': 3, 'title': 'meh', 'count': 11, 'perce...",20,...,0f0f0f,"[{'platform': {'id': 19, 'name': 'PS Vita', 's...","[{'platform': {'id': 2, 'name': 'PlayStation',...","[{'id': 4, 'name': 'Action', 'slug': 'action',...","[{'id': 4440, 'store': {'id': 3, 'name': 'Play...",NaN,"[{'id': 37796, 'name': 'exclusive', 'slug': 'e...","{'id': 1, 'name': 'Everyone', 'slug': 'everyone'}","[{'id': -1, 'image': 'https://media.rawg.io/me...",NaN
2,3935,breakquest-extra-evolution,BreakQuest: Extra Evolution,2012-10-16,False,https://media.rawg.io/media/screenshots/4e3/4e...,2.60,3,"[{'id': 3, 'title': 'meh', 'count': 10, 'perce...",20,...,0f0f0f,"[{'platform': {'id': 16, 'name': 'PlayStation ...","[{'platform': {'id': 2, 'name': 'PlayStation',...","[{'id': 11, 'name': 'Arcade', 'slug': 'arcade'...","[{'id': 4253, 'store': {'id': 3, 'name': 'Play...",NaN,"[{'id': 114, 'name': 'Physics', 'slug': 'physi...","{'id': 2, 'name': 'Everyone 10+', 'slug': 'eve...","[{'id': -1, 'image': 'https://media.rawg.io/me...",NaN
3,1991,killallzombies-2,#KILLALLZOMBIES,2014-10-28,False,https://media.rawg.io/media/screenshots/675/67...,2.36,1,"[{'id': 1, 'title': 'skip', 'count': 5, 'perce...",11,...,0f0f0f,"[{'platform': {'id': 1, 'name': 'Xbox One', 's...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 4, 'name': 'Action', 'slug': 'action',...","[{'id': 2061, 'store': {'id': 2, 'name': 'Xbox...",NaN,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...",NaN,"[{'id': -1, 'image': 'https://media.rawg.io/me...",NaN
4,979,tethered,Tethered,2016-10-25,False,https://media.rawg.io/media/screenshots/1e6/1e...,2.75,4,"[{'id': 4, 'title': 'recommended', 'count': 5,...",12,...,0f0f0f,"[{'platform': {'id': 18, 'name': 'PlayStation ...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 3, 'name': 'Adventure', 'slug': 'adven...","[{'id': 1002, 'store': {'id': 3, 'name': 'Play...",NaN,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...","{'id': 2, 'name': 'Everyone 10+', 'slug': 'eve...","[{'id': -1, 'image': 'https://media.rawg.io/me...",NaN


In [327]:
combined_api_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12140 entries, 0 to 12139
Data columns (total 30 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  12140 non-null  int64  
 1   slug                12140 non-null  object 
 2   name                12140 non-null  object 
 3   released            11881 non-null  object 
 4   tba                 12140 non-null  bool   
 5   background_image    12120 non-null  object 
 6   rating              12140 non-null  float64
 7   rating_top          12140 non-null  int64  
 8   ratings             12140 non-null  object 
 9   ratings_count       12140 non-null  int64  
 10  reviews_text_count  12140 non-null  int64  
 11  added               12140 non-null  int64  
 12  added_by_status     12140 non-null  object 
 13  metacritic          4628 non-null   float64
 14  playtime            12140 non-null  int64  
 15  suggestions_count   12140 non-null  int64  
 16  upda

In [328]:
metacritic_data.head()

,Title,released,Developer,Publisher,Genres,rating,User Score,ratings_count
0,ZIGGURAT,2012-02-17,ACTION BUTTON ENTERTAINMENT,FRESHUU INC.,ACTION,NaN,6.9,14
1,4X4 EVO 2,2001-11-15,TERMINAL REALITY,GATHERING,AUTO RACING SIM,RATED E FOR EVERYONE,0.0,0
2,MOTOGP 2,2002-01-22,NAMCO,NAMCO,AUTO RACING SIM,RATED E FOR EVERYONE,5.8,0
3,GOTHIC 3,2006-11-14,PIRANHA BYTES,ASPYR,WESTERN RPG,RATED T FOR TEEN,7.5,832
4,SIEGE SURVIVAL GLORIA VICTIS,2021-05-18,FISHTANKSTUDIO,BLACK EYE GAMES,RPG,NaN,6.5,10


In [329]:
metacritic_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14055 entries, 0 to 14054
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Title          14034 non-null  object 
 1   released       13991 non-null  object 
 2   Developer      13917 non-null  object 
 3   Publisher      13917 non-null  object 
 4   Genres         14034 non-null  object 
 5   rating         11005 non-null  object 
 6   User Score     14055 non-null  float64
 7   ratings_count  14055 non-null  int64  
dtypes: float64(1), int64(1), object(6)
memory usage: 878.6+ KB


## Transformations and merge

Function to extract platforms names

In [330]:
def extract_classification_names(json_str):
    # Check if the input is actually a string
    if isinstance(json_str, str):
        try:
            # Assuming the JSON strings are formatted with double quotes
            classification_data = json.loads(json_str.replace("'", '"'))
            # If the structure contains a list of platforms, we join them with a comma
            if isinstance(classification_data, list):
                classification_names = [classification['name'] for classification in classification_data]
                return ', '.join(classification_names)
            # If it's a dictionary and 'name' is the key for platform names
            elif 'name' in classification_data:
                return classification_data['name']
            else:
                return None
        except json.JSONDecodeError:
            return None
    else:
        return None

combined_api_data['rating'] = combined_api_data['esrb_rating'].apply(extract_classification_names)

# Check the transformation results
print(combined_api_data[['rating']].head())

         rating
0          None
1      Everyone
2  Everyone 10+
3          None
4  Everyone 10+


In [331]:
def extract_first_genre_name(json_str):
    if isinstance(json_str, str):
        try:
            # Correctly parse the JSON assuming single quotes in string representation
            genre_data = json.loads(json_str.replace("'", '"'))
            # Check if the data structure is a dictionary (single genre) or a list (multiple genres)
            if isinstance(genre_data, dict):
                # Return the name directly if it's a single genre
                return genre_data['name']
            elif isinstance(genre_data, list) and len(genre_data) > 0:
                # Return the name of the first genre if there are multiple
                return genre_data[0]['name']
            else:
                return None
        except json.JSONDecodeError:
            return None
    else:
        return None

# Apply the modified function to the 'genres' column
combined_api_data['genres'] = combined_api_data['genres'].apply(extract_first_genre_name)

# Display the result to verify
print(combined_api_data[['genres']])

          genres
0         Casual
1         Action
2         Arcade
3         Action
4      Adventure
...          ...
12135     Racing
12136     Action
12137  Adventure
12138        RPG
12139  Adventure

[12140 rows x 1 columns]


In [332]:
# Function to search for the word 'name' and extract the first platform name
def extract_first_platform_name_from_string(json_str, index):
    if not json_str or json_str.strip() in ['platforms', '']:
        print(f"Row {index}: Empty or placeholder string")
        return None
    
    # Check if the string is a plain text value
    if json_str.isalpha():
        print(f"Row {index}: Plain text value: {json_str}")
        return json_str
    
    try:
        # Replace single quotes with double quotes to form valid JSON
        json_str = json_str.replace("'", '"')
        
        # Find the platform names using regular expressions
        matches = re.findall(r'"name":\s*"([^"]+)"', json_str)
        if matches:
            return matches[0]  # Return the first match
        else:
            print(f"Row {index}: 'name' not found in {json_str}")
            return None
    except Exception as e:
        print(f"Row {index}: Error processing string: {e} - {json_str}")
        return None


# Apply the function to the DataFrame
combined_api_data['platforms'] = combined_api_data.apply(lambda row: extract_first_platform_name_from_string(row['platforms'], row.name), axis=1)

# Display the DataFrame
print(combined_api_data['platforms'])

Row 5699: 'name' not found in []
Row 8762: 'name' not found in []
0                   PC
1              PS Vita
2        PlayStation 3
3             Xbox One
4        PlayStation 4
             ...      
12135               PC
12136               PC
12137            macOS
12138               PC
12139               PC
Name: platforms, Length: 12140, dtype: object


In [333]:
combined_api_data = combined_api_data.map(lambda x: x.upper() if isinstance(x, str) else x)

In [335]:
combined_api_data.to_csv('../data/api_data2.csv', index=False)
print("Combined API data saved to 'api_data.csv'")

Combined API data saved to 'api_data.csv'


Genres in PostgreSQL are comma-separated strings, we will join API genres lists

In [334]:
api_relevant_columns = [
    'name', 'released', 'rating', 'ratings_count', 'metacritic', 'genres'
]
api_data_relevant = combined_api_data[api_relevant_columns].copy()

api_data_relevant['genres'] = api_data_relevant['genres'].apply(
    lambda x: ', '.join([genre['name'] for genre in eval(x)]) if pd.notnull(x) else x
)

NameError: name 'CASUAL' is not defined

Rename columns to match PostgreSQL data

In [ ]:
api_data_relevant.rename(columns={
    'name': 'Title',
    'rating': 'Rating',
    'metacritic': 'Metacritic Score',
    'ratings_count': 'Ratings Count',
    'genres': 'Genres'
}, inplace=True)

In [ ]:
# Standardize titles and release dates in db_data to ensure consistent formatting
metacritic_data['Title'] = metacritic_data['Title'].str.strip().str.upper()
metacritic_data['released'] = pd.to_datetime(metacritic_data['released']).dt.date

Merge the combined API data with the PostgreSQL data

In [ ]:
# Debug: Display the column names of both DataFrames
print("API Data Relevant Columns:", api_data_relevant.columns.tolist())
print("Database Data Columns:", metacritic_data.columns.tolist())

API Data Relevant Columns: ['Title', 'released', 'Rating', 'Ratings Count', 'Metacritic Score', 'Genres']
Database Data Columns: ['Title', 'released', 'Developer', 'Publisher', 'Genres', 'rating', 'User Score', 'ratings_count']


In [ ]:
# Merge the combined API data with the PostgreSQL data
merged_data = pd.merge(api_data_relevant, metacritic_data, on=['Title', 'released', 'Genres'], how='outer')

# Display merged data
print("Merged Data:")
print(merged_data.head())

Merged Data:
                                        Title    released  Rating  \
0                            "ANONYMOUS CODE"  2023-09-08     NaN   
1  "CHAOS HEAD NOAH  CHAOS CHILD DOUBLE PACK"  2022-10-07     NaN   
2                           "CHAOS HEAD NOAH"  2022-10-07     NaN   
3                      "DEATH END RE QUEST 2"  2020-08-25     NaN   
4                        "DEATH END RE QUEST"  2019-02-19     NaN   

   Ratings Count  Metacritic Score        Genres      Developer  \
0            NaN               NaN  VISUAL NOVEL            5PB   
1            NaN               NaN   COMPILATION         MAGES.   
2            NaN               NaN  VISUAL NOVEL         MAGES.   
3            NaN               NaN          JRPG  COMPILE HEART   
4            NaN               NaN          JRPG  COMPILE HEART   

        Publisher              rating  User Score  ratings_count  
0  SPIKE CHUNSOFT  RATED M FOR MATURE         7.8            6.0  
1  SPIKE CHUNSOFT  RATED M FOR MATU

In [ ]:
# Display rows with NaN values to understand the mismatches
nan_rows = merged_data[merged_data.isna().any(axis=1)]
print("Rows with NaN values:")
print(nan_rows)

Rows with NaN values:
                                            Title    released  Rating  \
0                                "ANONYMOUS CODE"  2023-09-08     NaN   
1      "CHAOS HEAD NOAH  CHAOS CHILD DOUBLE PACK"  2022-10-07     NaN   
2                               "CHAOS HEAD NOAH"  2022-10-07     NaN   
3                          "DEATH END RE QUEST 2"  2020-08-25     NaN   
4                            "DEATH END RE QUEST"  2019-02-19     NaN   
...                                           ...         ...     ...   
26190                                         NaN         NaT     NaN   
26191                                         NaN         NaT     NaN   
26192                                         NaN         NaT     NaN   
26193                                         NaN         NaT     NaN   
26194                                         NaN         NaT     NaN   

       Ratings Count  Metacritic Score        Genres      Developer  \
0                NaN          